In [1]:
# pip install azure-search-documents==11.6.b4

In [3]:
# Import required libraries  
import os  
import requests  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryType,
    VectorizedQuery,
    VectorFilterMode,    
)
from azure.search.documents.indexes.models import (  
 
    ExhaustiveKnnParameters,  
    ExhaustiveKnnAlgorithmConfiguration,
    FieldMapping,  
    HnswParameters,  
    HnswAlgorithmConfiguration,  
    InputFieldMappingEntry,  
    OutputFieldMappingEntry,  
    SimpleField,
    SearchField,  
    SearchFieldDataType,  
    SearchIndex,  
    SearchIndexer,  
    SearchIndexerDataContainer,  
    SearchIndexerDataSourceConnection,  
    SearchIndexerSkillset,  
    VectorSearch,  
    VectorSearchAlgorithmKind,  
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    WebApiSkill,
    SearchableField,
    
)  
from IPython.display import Image, display 
from azure.storage.blob import BlobServiceClient  

service_endpoint = "https://lge-search-prd-service.search.windows.net/" # cognitive search
#index_name = "testhjs" # index name+
api_version = "2023-11-01"
key = "eDGeShKFA1PcpgPDTT1DvgNQ7dPzj2TcRysQBuhoXTAzSeD7kJ32" #  cognitive search key
customSkill_endpoint = "https://test03984.azurewebsites.net/api/httptrigger2" # Azure Function Endpoint
blob_connection_string = "DefaultEndpointsProtocol=https;AccountName=lgestorprddocst;AccountKey=Ur9phTNrE90qrDLGmU061Jl2RnA5sNmDH4bYCadwc69UYCBdk4XFhAQKwmRv0+M632PCmggkZZPY+AStnaJKiw==;EndpointSuffix=core.windows.net" # Storage
container_name = "documents" # container name / foler name
credential = AzureKeyCredential(key)

In [2]:
service_endpoint = "https://lge-search-dev-service.search.windows.net/" # cognitive search
index_name = "testhjs" # index name+
api_version = "2023-11-01"
key = "aMKvczKGaybhbSy1CUFnPKGuzMbgBObCRJATPoyXYFAzSeA27WBx" #  cognitive search key
customSkill_endpoint = "https://test03984.azurewebsites.net/api/httptrigger2" # Azure Function Endpoint
blob_connection_string = "DefaultEndpointsProtocol=https;AccountName=lgeccsstorage;AccountKey=ipOLPwVxRTbng6kodSYLfPEv854wvppm1hMWeP2ISy0koXWxo0OSWtG5a5TcxafHnkWiWRYiVqeE+AStfAX6AQ==;EndpointSuffix=core.windows.net"
container_name = "preprocessed-data" # container name / foler name
credential = AzureKeyCredential(key)

# Connect to Blob Storage

Retreive your images from Blob Storage

In [25]:
# # Connect to Blob Storage
# blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
# container_client = blob_service_client.get_container_client(container_name)
# blobs = container_client.list_blobs()

# first_blob = next(blobs)
# blob_url = container_client.get_blob_client(first_blob).url
# print(f"URL of the first blob: {blob_url}")

# Connect your Blob Storage to a data source in Azure AI Search

In [26]:
# # Create a data source 
# ds_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))
# container = SearchIndexerDataContainer(name=container_name)
# data_source_connection = SearchIndexerDataSourceConnection(
#     name=f"{index_name}-blob",
#     type="azureblob",
#     connection_string=blob_connection_string,
#     container=container,
# )

In [7]:
index_name="uk-en"

In [8]:
from azure.search.documents.indexes.models import NativeBlobSoftDeleteDeletionDetectionPolicy

In [9]:
ds_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))
container = SearchIndexerDataContainer(name="documents", query="data/GB/English")
data_deletion_policy = NativeBlobSoftDeleteDeletionDetectionPolicy(
        odata_type="#Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy"
    )
data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-blob",
    type="azureblob",
    connection_string=blob_connection_string,
    container=container,
    dataDeletionDetectionPolicy=data_deletion_policy
)
data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'uk-en-blob' created or updated


# Create a skillset

Create a custom skill Azure Function which calls the Florence Vision API to generate image embeddings. See GetImageEmbeddings for details on the custom skill.

In [2]:
# Create a skillset  
skillset_name = "upload-data-to-index"
skill_uri = "https://lge-ai-prd-func.azurewebsites.net/aoai/api/upload_index"

# skill_uri = "https://indexer0226.azurewebsites.net/api/upload_index"
# skillset_name = "test-skillset"  
# skill_uri = "https://testljyindexer.azurewebsites.net/api/upload_index"
  
skill = WebApiSkill(
    uri=skill_uri,  
    inputs=[  
        InputFieldMappingEntry(name="file_path", source="/document/metadata_storage_path"),  
        InputFieldMappingEntry(name="recordId", source="/document/metadata_storage_name"),
    ],  
    outputs=[OutputFieldMappingEntry(name="type", target_name="type"),
             OutputFieldMappingEntry(name="iso_cd", target_name="iso_cd"),
             OutputFieldMappingEntry(name="language", target_name="language"),
             OutputFieldMappingEntry(name="product_group_code", target_name="product_group_code"),
             OutputFieldMappingEntry(name="product_code", target_name="product_code"),
             OutputFieldMappingEntry(name="product_model_code", target_name="product_model_code"),
             OutputFieldMappingEntry(name="data_id", target_name="data_id"),
             OutputFieldMappingEntry(name="mapping_key", target_name="mapping_key"),
             OutputFieldMappingEntry(name="chunk_num", target_name="chunk_num"),
             OutputFieldMappingEntry(name="pages", target_name="pages"),
             OutputFieldMappingEntry(name="url", target_name="url"),
             OutputFieldMappingEntry(name="title", target_name="title"),
             OutputFieldMappingEntry(name="main_text_vector", target_name="main_text_vector")],

    timeout="PT1M30S"
)  
  
skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to extract text vector",  
    skills=[skill],  
)  
  
client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  
client.create_or_update_skillset(skillset)  
print(f' {skillset.name} created')  


 upload-data-to-index created


## Create an index
Create your search index schema and vector search configuration:

In [22]:
skillset_name = "upload-data-to-index"
# skillset_name = "upload-data-to-index"

index_name = 'lv-lv'
data_source = 'lv-lv-blob'
directory_path = 'data/LV/Latvian/'

In [23]:
# Create a search index  
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)  
        # 필드명 정의
fields = [SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
          SearchableField(name="type", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="iso_cd", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="language", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="product_group_code", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="product_code", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="product_model_code", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="data_id", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="mapping_key", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="chunk_num", type=SearchFieldDataType.Int64, filterable=True, facetable=True),
          SearchableField(name="file_name", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="pages", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="url", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="title", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchableField(name="main_text_path", type=SearchFieldDataType.String, filterable=True, facetable=True),
          SearchField(name="main_text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                            searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile")]
  
# 백터 서치 configuration ( HNSW, KNN 사용 가능 )
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        )
    ]
)
  
# Create the search index with the vector search configuration  
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")  


lv-lv created


In [24]:
api_version = "2023-11-01"  

data_source_connection_payload = {
    "name": f"{index_name}-blob",
    "type": "azureblob",
    "credentials": {
        "connectionString": blob_connection_string    
    },
    "container": {
        "name": "documents",
        "query": directory_path
    },
    "dataChangeDetectionPolicy": None,
    "dataDeletionDetectionPolicy": {
        "@odata.type": "#Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy"
    }
}

# HTTP 헤더 구성
headers = {
    "Content-Type": "application/json",
    "api-key": key
}

# 데이터 소스 연결을 Azure Search에 생성 또는 업데이트
response = requests.put(
    f"{service_endpoint}/datasources/{index_name}-blob?api-version={api_version}",
    headers=headers,
    json=data_source_connection_payload
)

# 요청이 성공했는지 확인
if response.status_code == 200 or response.status_code == 201 or response.status_code == 204:
    print(f"Data source '{data_source_connection_payload['name']}' created or updated successfully.")
else:
    print("Failed to create or update data source. Response:", response)

Data source 'lv-lv-blob' created or updated successfully.


## Create an indexer

Create or update an indexer to process images and populate the search index

In [25]:
# Create an indexer  
indexer_name = f"{index_name}-indexer"
indexer = SearchIndexer(  
    name=indexer_name,  
    description=f"{index_name} Indexer",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source,
    parameters={
        "configuration": {
            "executionEnvironment": "private"
        }
    },
    field_mappings=[
        FieldMapping(source_field_name="metadata_storage_path", target_field_name="main_text_path"),  
        FieldMapping(source_field_name="metadata_storage_name", target_field_name="file_name")  
    ],  
    output_field_mappings=[
        FieldMapping(source_field_name="/document/type", target_field_name="type"),
        FieldMapping(source_field_name="/document/iso_cd", target_field_name="iso_cd"),
        FieldMapping(source_field_name="/document/language", target_field_name="language"),
        FieldMapping(source_field_name="/document/product_group_code", target_field_name="product_group_code"),
        FieldMapping(source_field_name="/document/product_code", target_field_name="product_code"),
        FieldMapping(source_field_name="/document/product_model_code", target_field_name="product_model_code"),
        FieldMapping(source_field_name="/document/data_id", target_field_name="data_id"),
        FieldMapping(source_field_name="/document/mapping_key", target_field_name="mapping_key"),
        FieldMapping(source_field_name="/document/chunk_num", target_field_name="chunk_num"),
        FieldMapping(source_field_name="/document/pages", target_field_name="pages"),
        FieldMapping(source_field_name="/document/url", target_field_name="url"),
        FieldMapping(source_field_name="/document/title", target_field_name="title"),
        FieldMapping(source_field_name="/document/main_text_vector", target_field_name="main_text_vector")  
    ]  
)  
  
indexer_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  
indexer_result = indexer_client.create_or_update_indexer(indexer)  

In [8]:
indexer_name = 'ee-et-indexer'

In [9]:
# Run the indexer
indexer_client.run_indexer(indexer_name)
print(f' {indexer_name} running')

 pt-pt-indexer running


In [10]:
from ai_search import AISearch
search_instance = AISearch()
indexer_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  

In [11]:
index_name='pt-pt'

In [12]:
status = indexer_client.get_indexer_status(name=f"{index_name}-indexer")
print("ㅁ num_documents :",search_instance.get_num_documents(index_name=index_name))
print("ㅁ status :",status.execution_history[0].status)
print("ㅁ start_time :",status.execution_history[0].start_time)
print("ㅁ end_time :",status.execution_history[0].end_time)
print("ㅁ item_count :",status.execution_history[0].item_count)
print("ㅁ failed_item_count :",status.execution_history[0].failed_item_count)

인덱스 'pt-pt'의 문서 수: 0
ㅁ num_documents : 0


IndexError: list index out of range

In [15]:
indexer_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)  

In [16]:
print(f"\nㅁIndexes\n{[index for index in index_client.list_index_names()]}]\n")
print(f"\nㅁSkillsets\n{indexer_client.get_skillset_names()}\n")
print(f"\nㅁIndexers\n{indexer_client.get_indexer_names()}\n")
print(f"\nㅁDataSources\n{indexer_client.get_data_source_connection_names()}\n")


ㅁIndexes
['at-de', 'ch-de', 'ch-fr', 'de-de', 'dk-da', 'es-es', 'fi-fi', 'fr-fr', 'it-it', 'no-no', 'pl-pl', 'se-sv', 'uk-en']]


ㅁSkillsets
['upload-data-to-index']


ㅁIndexers
['at-de-indexer', 'ch-de-indexer', 'ch-fr-indexer', 'de-de-indexer', 'dk-da-indexer', 'es-es-indexer', 'fi-fi-indexer', 'fr-fr-indexer', 'it-it-indexer', 'no-no-indexer', 'pl-pl-indexer', 'se-sv-indexer', 'uk-en-indexer']


ㅁDataSources
['at-de-blob', 'ch-de-blob', 'ch-fr-blob', 'de-de-blob', 'dk-da-blob', 'es-es-blob', 'es-es-new-blob', 'fi-fi-blob', 'fr-fr-blob', 'it-it-blob', 'no-no-blob', 'pl-pl-blob', 'se-sv-blob', 'uk-en-blob']



In [ ]:
result = search_instance.get_documents(index_name='de-de', type='mapping_key',mapping_key='')

In [ ]:
indexer_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  
indexer_result = indexer_client.create_or_update_indexer(indexer)  